## Prepare 

In [1]:
import pandas as pd
%pylab inline
from datetime import datetime
from datetime import timedelta

# date handling
dateparse1 = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
dateparse2 = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')

Populating the interactive namespace from numpy and matplotlib


## Load Data

In [21]:
# Page Impressions
piDat = pd.read_csv('../data/PI_angereicherteDaten.csv.txt',sep='\t',low_memory=False,na_values=['-'])
# Visits
vDat = pd.read_csv('../data/V_angereicherteDaten.txt',sep='\t',low_memory=False,na_values=['-'])


# contract data from contracts which are of interest to us
contract = pd.read_pickle('contract_data') 

In [22]:
homePi= pd.read_csv('../data/WT_Pis_Home.csv',header=8,  sep = "\t")
homePi.drop(columns='PI [Home] ', inplace=True)
homePi.columns = ['Wochen', 'PI [Home]']
homePi['URM - Eigene Besucher-ID']  = homePi.index
piDat = pd.merge(piDat, homePi, on = ['URM - Eigene Besucher-ID', 'Wochen'])
piDat.head()

,URM - Eigene Besucher-ID,Wochen,Visits,Visits Plus,Page Impressions,Page Impressions plus,PI [mit Komment],PI [Typ: Section],PI [Typ: Article],PI Plus [mit Komment],PI Plus [Typ: Section],PI Plus [Typ: Article],PI [Home]
0,ASAG1644091,2018/10,1,1.0,10,3.0,NaN,NaN,7.0,NaN,NaN,3.0,2.0
1,ASAG1644091,2018/14,1,1.0,7,2.0,NaN,1.0,3.0,NaN,NaN,2.0,1.0
2,ASAG1644223,2018/15,1,1.0,9,3.0,NaN,1.0,4.0,NaN,NaN,3.0,3.0
3,ASAG1644393,2017/51,1,NaN,19,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0
4,ASAG1644396,2018/12,1,NaN,2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0


In [3]:
homePi= pd.read_csv('../data/WT_Pis_Home.csv',header=8,  sep = "\t")
homePi.drop(columns='PI [Home] ', inplace=True)
homePi.columns = ['Wochen', 'PI [Home]']
homePi['URM - Eigene Besucher-ID']  = homePi.index

piDat = pd.merge(piDat, homePi, on = ['URM - Eigene Besucher-ID', 'Wochen'])

#piDat.head()
#sum(homePi['Wochen'].isna())

In [4]:
print(piDat.columns.values)
print(vDat.columns.values)

['URM - Eigene Besucher-ID' 'Wochen' 'Visits' 'Visits Plus'
 'Page Impressions' 'Page Impressions plus' 'PI [mit Komment]'
 'PI [Typ: Section]' 'PI [Typ: Article] ' 'PI Plus [mit Komment]'
 'PI Plus [Typ: Section]' 'PI Plus [Typ: Article]' 'PI [Home]']
['URM - Eigene Besucher-ID' 'Wochen' 'Visits' 'Visits Plus'
 'Page Impressions' 'Page Impressions plus' 'Visits [mit Komment]'
 'Visits [mit Section] ' 'Visits [Mobile] ' 'Visits [Desktop] '
 'Visits [mit Article] ' 'Visits Plus [mit Komment]'
 'Visits Plus [mit Section]' 'Visits Plus [Mobile]'
 'Visits Plus [Desktop]' 'Visits Plus [mit Article]']


### Make PivotTables & Filter for Subscriptions of interest

In [30]:
# from piDat and vDat make pivot tables for each KPI of interest such that each line represents one user, 
# columns represent weeks
#
# filter for subscriptions of interest ('Vollabo' & WeltPlus etc., no strange cancellation reasons)
# = 
#

# lists of variable names for new pivot tbales and lists of colnumns names in original data
list_pi_varNames = ['v','vPlus',
                    'pi','piPlus','piComment',
                    'piSection','piArticle','piPlusComment',
                    'piPlusSection','piPlusArticle', 'piHome']
list_pi_colNames = ['Visits', 'Visits Plus',
 'Page Impressions', 'Page Impressions plus' ,'PI [mit Komment]',
 'PI [Typ: Section]', 'PI [Typ: Article] ', 'PI Plus [mit Komment]',
 'PI Plus [Typ: Section]' ,'PI Plus [Typ: Article]', 'PI [Home]']

list_v_varNames = ['vComment','vSection','vArticle','vMobile','vDesktop',
                   'vPlusComment','vPlusArticle','vPlusMobile','vPlusDesktop']
list_v_colNames = ['Visits [mit Komment]', 'Visits [mit Section] ','Visits [mit Article] ', 
                   'Visits [Mobile] ', 'Visits [Desktop] ', 
                   'Visits Plus [mit Komment]', 'Visits Plus [mit Article]',
                   'Visits Plus [Mobile]', 'Visits Plus [Desktop]']

# list of IDs from subscriptions of interest
conIdx = contract['WT SSOID']


# contains in the end all pivot tables
pivot_tables_pi_v = {}

# from piDat
for idx, val in enumerate(list_pi_varNames):
    # make pivot table
    table = pd.pivot_table(piDat,values=list_pi_colNames[idx], 
                                  index='URM - Eigene Besucher-ID', columns='Wochen',aggfunc='first')
    # filter
    pivot_tables_pi_v[val] = table.loc[table.index.isin(conIdx)]
    
# fom vDat
for idx, val in enumerate(list_v_varNames):
    # make pivot table
    table = pd.pivot_table(vDat, values=list_v_colNames[idx], 
                                  index='URM - Eigene Besucher-ID', columns='Wochen',aggfunc='first')
    # filter
    pivot_tables_pi_v[val] = table.loc[table.index.isin(conIdx)]

    
# make relations with v
list_total = list_pi_varNames[1:]+list_v_varNames
for idx, val in enumerate(list_total):
    # name to save table
    table_name= val + '_per_v'
    # make relations
    pivot_tables_pi_v[table_name] = pivot_tables_pi_v[val]/pivot_tables_pi_v['v']


Page immpression Plus Section & Visits Plus Section haben keine Einträge. 

### Metriken berechnen

In [13]:
pivot_tables_pi_v['v'].columns.values

array(['2017/51', '2017/52', '2018/01', '2018/02', '2018/03', '2018/04',
       '2018/05', '2018/06', '2018/07', '2018/08', '2018/09', '2018/10',
       '2018/11', '2018/12', '2018/13', '2018/14', '2018/15', '2018/16',
       '2018/17', '2018/18', '2018/19', '2018/20', '2018/21'],
      dtype=object)

In [34]:
# import function
import os
sys.path.append(os.path.abspath("/Users/slueck/OneDrive - Axel Springer SE/Dokumente/pylibs/"))
import calcMetrics

In [35]:
for key, value in pivot_tables_pi_v.items():
    print(key)


pivot_pi_v_metrics ={}
for key, value in pivot_tables_pi_v.items():
    
    print(datetime.now().time())
    print(key)
    print('')
    
    # column names
    name = key + '_'
    
    pivot_pi_v_metrics[key] = calcMetrics.apply_calc_metrics_on_table( table = pivot_tables_pi_v[key], name = name, 
                                                                      contract_dat=contract,
                                                                      calc_user_behav_before_canc = True,
                                                                      real_last_behavior = True)

v
vPlus
pi
piPlus
piComment
piSection
piArticle
piPlusComment
piPlusSection
piPlusArticle
piHome
vComment
vSection
vArticle
vMobile
vDesktop
vPlusComment
vPlusArticle
vPlusMobile
vPlusDesktop
vPlus_per_v
pi_per_v
piPlus_per_v
piComment_per_v
piSection_per_v
piArticle_per_v
piPlusComment_per_v
piPlusSection_per_v
piPlusArticle_per_v
piHome_per_v
vComment_per_v
vSection_per_v
vArticle_per_v
vMobile_per_v
vDesktop_per_v
vPlusComment_per_v
vPlusArticle_per_v
vPlusMobile_per_v
vPlusDesktop_per_v
12:06:43.681737
v



/Users/slueck/OneDrive - Axel Springer SE/Dokumente/pylibs/calcMetrics.py:184: RuntimeWarning: invalid value encountered in double_scalars
  dat['zsc_usr_behav1'] = (dat['usr_behav1'] - mean) / sd
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1018: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/slueck/OneDrive - Axel Springer SE/Dokumente/pylibs/calcMetrics.py:182: RuntimeWarning: invalid value encountered in double_scalars
  dat['zsc_usr_behav3'] = (dat['usr_behav3'] - mean) / sd
/Users/slueck/OneDrive - Axel Springer SE/Dokumente/pylibs/calcMetrics.py:181: RuntimeWarning: invalid value encountered in double_scalars
  dat['zsc_usr_behav4'] = (dat['usr_behav4'] - mean) / sd
/Users/slueck/OneDrive - Axel Springer SE/Dokumente/pylibs/calcMetrics.py:183: RuntimeWarning: invalid value encountered in double_scalars
  dat['zsc_usr_behav2'] = (dat['usr_behav2'] - mean) / sd


12:08:33.482881
vPlus

12:10:21.836709
pi

12:12:13.984483
piPlus

12:14:02.668135
piComment

12:15:52.269837
piSection

12:17:41.838333
piArticle

12:19:32.215345
piPlusComment

12:21:22.618092
piPlusSection

12:21:22.618546
piPlusArticle

12:23:13.406821
piHome

12:25:04.157277
vComment

12:27:09.659131
vSection

12:29:15.259847
vArticle

12:31:17.790966
vMobile

12:33:22.287915
vDesktop

12:35:28.066384
vPlusComment

12:37:33.246010
vPlusArticle

12:39:40.292100
vPlusMobile

12:41:44.116313
vPlusDesktop

12:43:47.893023
vPlus_per_v

12:45:34.317036
pi_per_v

12:47:30.333648
piPlus_per_v

12:49:29.366103
piComment_per_v

12:51:28.996650
piSection_per_v

12:53:20.153180
piArticle_per_v

12:55:06.334199
piPlusComment_per_v

12:56:52.064866
piPlusSection_per_v

12:58:37.591330
piPlusArticle_per_v

13:00:23.166085
piHome_per_v

13:02:08.745990
vComment_per_v

13:04:08.978091
vSection_per_v

13:06:09.929244
vArticle_per_v

13:08:10.912380
vMobile_per_v

13:10:11.681991
vDesktop_per_v

13:

### Join all tables in order to save them

In [39]:
# initialize
all_tables = pivot_pi_v_metrics['v']

# remove 'v' in order to iterate through remaining dict
pivot_pi_v_metrics.pop('v')

for key, val in pivot_pi_v_metrics.items():
    all_tables = all_tables.join(pivot_pi_v_metrics[key])

# include length of contract in months
contract['Vertragslaenge'] = round((contract['VE Tag']-contract['VB Tag'])/ np.timedelta64(1, 'M'))

# join with contract data
contract.index = contract['WT SSOID']
all_tables_cont = contract[['WT SSOID', 'VE Jahr/KW', 'Kuendigungsstatus', 'Kuendigungsgrund',
       'Promotion ID', 'Vertragsform', 'Trial Kennzeichen', 'Vertragslaenge',
       'Vertragsphase', 'VB Tag', 'VB Jahr', 'VB KW', 'VB Jahr/KW',
       'VE Tag', 'VE Jahr', 'VE KW',
       'K Tag']].join(all_tables)

KeyError: 'v'

In [37]:
pivot_pi_v_metrics['piHome']

,piHome_2017/36,piHome_2017/37,piHome_2017/38,piHome_2017/39,piHome_2017/40,piHome_2017/41,piHome_2017/42,piHome_2017/43,piHome_2017/44,piHome_2017/45,...,piHome_sd,piHome_sd_normed,piHome_usr_behav1,piHome_usr_behav2,piHome_usr_behav3,piHome_usr_behav4,piHome_zsc_usr_behav1,piHome_zsc_usr_behav2,piHome_zsc_usr_behav3,piHome_zsc_usr_behav4
URM - Eigene Besucher-ID,,,,,,,,,,,,,,,,,,,,,
ASAG1644540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.720775,0.847319,NaN,NaN,3.0,NaN,NaN,NaN,-0.544705,NaN
ASAG1645650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.707107,0.282843,NaN,NaN,2.0,NaN,NaN,NaN,-0.707107,NaN
ASAG1648931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.874775,0.630464,2.0,19.0,10.0,17.0,-1.245696,1.648032,0.116059,1.307593
ASAG1652928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.952138,0.600124,9.0,51.0,44.0,57.0,-1.236773,0.767795,0.433701,1.054162
ASAG1663079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.017032,0.893153,3.0,2.0,4.0,11.0,-0.621044,-0.787239,-0.454849,0.708515
ASAG1666229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.067843,0.450348,19.0,18.0,18.0,25.0,0.910755,0.745951,0.745951,1.899574
ASAG1666305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.848547,0.273445,NaN,8.0,27.0,28.0,NaN,-2.488915,0.285396,0.431412
ASAG1668230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.078793,0.322288,5.0,24.0,40.0,47.0,-2.688866,-1.115862,0.208774,0.788302
ASAG1668634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.848967,0.895631,16.0,16.0,54.0,73.0,-0.267678,-0.267678,1.748348,2.756360


## Save Data

In [38]:
all_tables_cont.to_pickle('PIV_angereichert_metrics_wLastUserBehav')